In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import svm
from sklearn import naive_bayes

In [2]:
user_genre_rate = pd.read_csv('dataset/users_genres_rating.csv')
top_animes = pd.read_csv('dataset/top_animes_complete.csv')

top_animes

,MAL_ID,completed_user_ids
0,16498,"189037|6, 162615|10, 68042|9, 283786|9, 259790..."
1,1535,"189037|9, 162615|10, 68042|10, 283786|10, 2597..."
2,11757,"189037|7, 162615|10, 68042|4, 283786|9, 259790..."
3,30276,"189037|8, 162615|10, 68042|9, 283786|9, 259790..."
4,31964,"189037|7, 162615|10, 68042|8, 283786|8, 259790..."
...,...,...
195,6347,"189037|8, 162615|10, 68042|9, 283786|9, 259790..."
196,11633,"189037|7, 162615|10, 68042|8, 283786|8, 259790..."
197,33506,"189037|7, 162615|10, 68042|8, 283786|8, 259790..."
198,37520,"189037|8, 68042|9, 283786|10, 259790|8, 291207..."


In [3]:
user_genre_rate

,user_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,189037,6.995090,7.008240,6.974138,7.005295,6.823377,6.922018,6.980873,7.009602,6.994641,...,7.029516,6.993506,6.992481,7.008621,6.968397,6.921053,7.000000,6.806202,6.416667,6.78125
1,162615,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000
2,68042,6.133354,6.352613,5.881720,6.059456,5.532951,6.019753,6.150605,5.374648,6.076405,...,6.125761,6.062645,6.294023,6.114603,6.090909,6.663551,5.305556,5.791667,4.500000,4.62500
3,283786,6.991761,6.892308,6.766667,6.776759,6.170787,6.898876,7.001941,6.631732,6.793103,...,6.831423,6.960280,7.042403,7.031311,7.070434,7.439655,6.378378,7.191667,3.000000,6.00000
4,259790,5.585144,5.640308,4.486486,5.257695,3.876543,5.528169,6.159873,5.056338,5.366282,...,5.375774,5.569322,4.805732,5.501062,5.923307,6.744681,3.130435,5.462264,4.920000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,123966,7.401460,7.488584,8.000000,7.320225,7.250000,7.700000,7.665541,6.908730,7.367953,...,7.531073,8.000000,7.650000,7.439394,7.506276,7.818182,0.000000,7.173913,0.000000,0.00000
1996,31159,7.493438,7.458874,7.600000,7.229985,6.833333,7.416667,7.610959,6.644231,7.274247,...,7.403409,8.000000,7.411765,7.515625,7.547529,8.166667,0.000000,7.611111,0.000000,0.00000
1997,161305,7.159605,7.332226,8.076923,7.050562,7.461538,7.321429,7.674556,6.327684,7.099398,...,7.417323,7.530973,7.313725,7.116071,7.250000,7.710526,0.000000,7.318182,0.000000,0.00000
1998,36853,9.975771,9.994135,10.000000,9.992026,9.880000,10.000000,9.993258,10.000000,9.978648,...,10.000000,10.000000,9.950820,9.976744,10.000000,10.000000,0.000000,10.000000,10.000000,0.00000


In [4]:
def init_dict():
    result = dict()
    result["mean_error"] = 0
    result["score"] = 0
    
    return result
    
linear_regression_dict = init_dict()
svm_dict = init_dict()
naive_dict = init_dict()

In [5]:
def handle_one_model(model, store_dict, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    store_dict["mean_error"] = store_dict['mean_error'] + mean_squared_error(y_test,model.predict(X_test))
    
    store_dict["score"] = store_dict["score"] + model.score(X_test,y_test)

In [6]:
def compute_animes(uid_str):
    uid_str_list = uid_str.split(", ")
    
    uid_score = dict()
    uid_list = []
    
    for pair in uid_str_list:
        s_pair = pair.split("|")
        uid = int(s_pair[0])
        score = int(s_pair[1])
        
        uid_score[uid] = score
        uid_list.append(uid)
    
    clean_user_genre_rate = user_genre_rate[user_genre_rate['user_id'].isin(uid_list)]
    
    X_list = []
    y_list = []
    
    for index, row in clean_user_genre_rate.iterrows():
        uid = int(row[0])
        genres = row[1:]
        temp = []
        
        for i in range(0,44):
            temp.append(genres[i])
        
        arr = np.array(temp)
        
        X_list.append(arr)
        y_list.append(uid_score[uid])
    
    X = np.array(X_list)
    y = np.array(y_list)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    lr = linear_model.LinearRegression()
    handle_one_model(lr, linear_regression_dict, X_train, X_test, y_train, y_test)

    
    svc = svm.SVC()
    handle_one_model(svc, svm_dict, X_train, X_test, y_train, y_test)
    
    
    nb = naive_bayes.GaussianNB()
    handle_one_model(nb, naive_dict, X_train, X_test, y_train, y_test)
    
    
    

In [7]:
count = 0
for index, row in top_animes.iterrows():
    mal_id = row['MAL_ID']
    compute_animes(row['completed_user_ids'])
    count += 1

In [8]:
def output(store_dict, name):
    print("Using " + name + ":")
    print("average mean squared error:  " + str(store_dict["mean_error"] / count)) 
    print("average score:  " + str(store_dict["score"] / count))

In [9]:
output(linear_regression_dict, "Linear Regression")

Using Linear Regression:
average mean squared error:  1.3197434036169529
average score:  0.4601176749106134


In [10]:
output(svm_dict, "Super Vector Machines")

Using Super Vector Machines:
average mean squared error:  1.9305788421583743
average score:  0.36097905973615296


In [11]:
output(naive_dict, "Naive Bayes")

Using Naive Bayes:
average mean squared error:  3.028555791195076
average score:  0.30207910765976664
